In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Data preprocessing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Loading the datasets
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)

# Neural Network Definition
class MLP(nn.Module):
    def __init__(self, layer_sizes):
        super(MLP, self).__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))

    def forward(self, x):
        x = x.view(-1, 28 * 28)  # Flatten the image
        for layer in self.layers[:-1]:
            x = torch.relu(layer(x))
        x = self.layers[-1](x)
        return x

# Experiment configurations
configurations = [
    # Your configurations
]

import csv

# Train and evaluate each configuration
results = []
for idx, config in enumerate(configurations):
    train_loader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=config["batch_size"], shuffle=False)

    model = MLP(config["layer_sizes"])

    if config["optimizer"] == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=config["lr"])
    elif config["optimizer"] == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=config["lr"])
    elif config["optimizer"] == "RMSprop":
        optimizer = optim.RMSprop(model.parameters(), lr=config["lr"])

    criterion = nn.CrossEntropyLoss()
    num_epochs = 25  # Adjust the number of epochs for detailed analysis

    # Training loop
    for epoch in range(num_epochs):
        model.train()
        for data in train_loader:
            inputs, labels = data
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        # Evaluation after each epoch
        correct = 0
        total = 0
        model.eval()
        with torch.no_grad():
            for data in test_loader:
                images, labels = data
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total

        # Append results for each epoch
        if epoch == num_epochs - 1:  # If last epoch, add the result to the list
            results.append({
                "Configuration": idx + 1,
                "Layer Sizes": config["layer_sizes"],
                "Optimizer": config["optimizer"],
                "Learning Rate": config["lr"],
                "Batch Size": config["batch_size"],
                "Epoch": epoch + 1,
                "Accuracy": f'{accuracy:.2f}%'
            })

# Write results to a CSV file
with open('final_results.csv', mode='w', newline='') as file:
    fieldnames = ['Configuration', 'Layer Sizes', 'Optimizer', 'Learning Rate', 'Batch Size', 'Epoch', 'Accuracy']
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    writer.writeheader()
    for result in results:
        writer.writerow(result)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from keras.optimizers import Adam, SGD
from keras.utils import to_categorical

# Load and preprocess the data
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Function to create a CNN model
def create_cnn_model(num_conv_layers, filters, kernel_size, pool_size, dropout_rate, optimizer):
    model = Sequential()
    model.add(Conv2D(filters[0], kernel_size, activation='relu', input_shape=(28, 28, 1)))

    # Reduce downsampling by adjusting pool size or removing some pooling layers
    model.add(MaxPooling2D(pool_size=(1, 1)))  # Adjust pool size as needed

    for i in range(1, num_conv_layers):
        model.add(Conv2D(filters[i], kernel_size, activation='relu'))
        model.add(MaxPooling2D(pool_size=pool_size))  # Adjust pool size as needed

    model.add(Flatten())
    model.add(Dropout(dropout_rate))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Different configurations to try
configs = [
    (3, [32, 64, 128], (3, 3), (2, 2), 0.3, Adam(learning_rate=0.001)),
    (2, [64, 128], (3, 3), (2, 2), 0.3, SGD(learning_rate=0.01)),
    (3, [32, 64, 128], (3, 3), (2, 2), 0.3, Adam(learning_rate=0.001)),
    (2, [64, 128], (3, 3), (2, 2), 0.3, SGD(learning_rate=0.01)),
    (3, [32, 64, 128], (3, 3), (2, 2), 0.3, SGD(learning_rate=0.001)),
    (2, [64, 128], (3, 3), (2, 2), 0.3, Adam(learning_rate=0.0005)),
    (4, [32, 64, 128, 256], (3, 3), (2, 2), 0.4, Adam(learning_rate=0.001)),
    (3, [64, 128, 256], (3, 3), (2, 2), 0.4, SGD(learning_rate=0.001)),
    (2, [128, 256], (3, 3), (2, 2), 0.4, Adam(learning_rate=0.0005)),
    (4, [64, 128, 256, 512], (3, 3), (2, 2), 0.5, SGD(learning_rate=0.001)),
    (3, [32, 64, 128], (5, 5), (2, 2), 0.3, Adam(learning_rate=0.0001)),
    (2, [64, 128], (5, 5), (2, 2), 0.3, SGD(learning_rate=0.001)),
    (3, [32, 64, 128], (5, 5), (2, 2), 0.3, SGD(learning_rate=0.0005)),
    (2, [64, 128], (5, 5), (2, 2), 0.3, Adam(learning_rate=0.0001)),
    (4, [32, 64, 128, 256], (5, 5), (2, 2), 0.4, Adam(learning_rate=0.0005)),
    (3, [64, 128, 256], (5, 5), (2, 2), 0.4, SGD(learning_rate=0.0005)),
    (2, [128, 256], (5, 5), (2, 2), 0.4, Adam(learning_rate=0.0001)),
    (4, [64, 128, 256, 512], (5, 5), (2, 2), 0.5, SGD(learning_rate=0.0005)),
    (3, [32, 64, 128], (3, 3), (3, 3), 0.3, Adam(learning_rate=0.0001)),
    (2, [64, 128], (3, 3), (3, 3), 0.3, SGD(learning_rate=0.001)),
]

# Evaluate each configuration
for config in configs:
    model = create_cnn_model(*config)
    model.fit(x_train, y_train, epochs=10, batch_size=64, validation_split=0.2, verbose=1)
    loss, accuracy = model.evaluate(x_test, y_test)
    print(f'Test accuracy: {accuracy:.2f}')


Epoch 1/10
750/750 [==============================] - 147s 195ms/step - loss: 0.4690 - accuracy: 0.8312 - val_loss: 0.3292 - val_accuracy: 0.8811
Epoch 2/10
750/750 [==============================] - 144s 192ms/step - loss: 0.2972 - accuracy: 0.8923 - val_loss: 0.2713 - val_accuracy: 0.8997
Epoch 3/10
750/750 [==============================] - 145s 193ms/step - loss: 0.2497 - accuracy: 0.9091 - val_loss: 0.2422 - val_accuracy: 0.9108
Epoch 4/10
750/750 [==============================] - 141s 188ms/step - loss: 0.2182 - accuracy: 0.9191 - val_loss: 0.2156 - val_accuracy: 0.9244
Epoch 5/10
750/750 [==============================] - 139s 185ms/step - loss: 0.1958 - accuracy: 0.9278 - val_loss: 0.2168 - val_accuracy: 0.9233
Epoch 6/10
750/750 [==============================] - 139s 186ms/step - loss: 0.1728 - accuracy: 0.9357 - val_loss: 0.2182 - val_accuracy: 0.9213
Epoch 7/10
750/750 [==============================] - 140s 186ms/step - loss: 0.1584 - accuracy: 0.9406 - val_loss: 0.2150 -